In [1]:
from app import app, db, User, Question, Submission
import pandas as pd
import json

In [2]:
df = pd.read_csv("/home/anubhab/Documents/Semester 7/BTP/ML-SurveyTool-Flask/survey_input_DD_cc_only-test (6).csv")
data = df.to_json("test_data.json",orient="records")

In [3]:
with open('test_data.json', 'r') as f:
    data = json.load(f)

In [4]:
data[0]

{'id': 783,
 'con': '<div class="utterance"><b>Speaker 1:</b> Hello . Can I help you , ma\'am ?  </div><div class="utterance"><b>Speaker 2:</b>  Yes . Could you tell me how to send some money to someone with an account with a bank in Germany ?  </div>',
 'response_A': '<div class="utterance"><b>Speaker 1:</b> Yes, please go to the bank and fill out this form. </div>',
 'response_A_raw': 'Yes, please go to the bank and fill out this form.',
 'response_A_source': 'DialogRPT-new',
 'response_B': '<div class="utterance"><b>Speaker 1:</b> Well , you can pay by mail , telegraphic or SWIFT transfer . Or you can have a banker\'s draft and mail it yourself to the beneficiary .  </div>',
 'response_B_raw': "Well , you can pay by mail , telegraphic or SWIFT transfer . Or you can have a banker's draft and mail it yourself to the beneficiary . ",
 'response_B_source': 'gt'}

In [5]:
with app.app_context():
    db.create_all()
    for row in data:
        # print(row)
        q = Question(
            sample_id=row['id'],
            context=json.dumps(row['con']),
            response_a=row['response_A_raw'],
            response_b=row['response_B_raw'],
            response_a_src=row['response_A_source'],
            response_b_src=row['response_B_source'],
            is_active=True
        )
        db.session.add(q)
    db.session.commit()

In [6]:
with app.app_context():
    demo_user = User(
        prolific_pid="user2",
        ref_url="https://localhost"
    )
    print(demo_user)
    db.session.add(demo_user)
    db.session.commit()

User('user2', 'https://localhost')


In [7]:
with app.app_context():
    print(User.query.all())

[User('user2', 'https://localhost')]


In [8]:
with app.app_context():
    print(Question.query.all())

[Question('sid:783', 'DialogRPT-new' vs. 'gt', '2023-10-13 12:21:22.389913', active:'True'), Question('sid:4583', 'gt' vs. 'CORAL.BB-BERT-mixp', '2023-10-13 12:21:22.389915', active:'True'), Question('sid:1478', 'gt' vs. 'DialogRPT-new', '2023-10-13 12:21:22.389916', active:'True'), Question('sid:3578', 'gt' vs. 'CORAL.BB-BERT-mixp', '2023-10-13 12:21:22.389916', active:'True'), Question('sid:69', 'DialogRPT-new' vs. 'gt', '2023-10-13 12:21:22.389918', active:'True'), Question('sid:5401', 'gt' vs. 'Blenderbot-400M-ft', '2023-10-13 12:21:22.389919', active:'True'), Question('sid:634', 'gt' vs. 'DialogRPT-new', '2023-10-13 12:21:22.389919', active:'True'), Question('sid:4095', 'Blenderbot-400M-ft' vs. 'gt', '2023-10-13 12:21:22.389919', active:'True'), Question('sid:3697', 'Blenderbot-400M-ft' vs. 'gt', '2023-10-13 12:21:22.389920', active:'True'), Question('sid:3451', 'gt' vs. 'Blenderbot-400M-ft', '2023-10-13 12:21:22.389920', active:'True'), Question('sid:5401', 'gt' vs. 'DialogRPT-ne

In [28]:
import sqlite3
import pandas as pd

In [29]:
conn = sqlite3.connect('instance/site.db')

In [30]:
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = [table[0] for table in tables]

print(table_names)

['user', 'question', 'submission']


In [31]:
table_data = {}
table_name = "submission"
query = f"SELECT * FROM {table_name};"
table_data[table_name] = pd.read_sql_query(query, conn)

table_data["submission"].columns

Index(['id', 'date_added', 'submission_json', 'user_id', 'question_id',
       'is_submitted'],
      dtype='object')

In [35]:
table_data['submission']

,id,date_added,submission_json,user_id,question_id,is_submitted
0,1,2023-10-16 20:21:21.414529,{},2,1,0
1,2,2023-10-16 20:21:21.414531,{},2,2,0
2,3,2023-10-16 20:21:21.414533,{},2,3,0
3,4,2023-10-16 20:21:21.414534,{},2,4,0
4,5,2023-10-16 20:21:21.414534,{},2,5,0
...,...,...,...,...,...,...
145,146,2023-10-16 20:21:21.414571,{},2,146,0
146,147,2023-10-16 20:21:21.414571,{},2,147,0
147,148,2023-10-16 20:21:21.414571,{},2,148,0
148,149,2023-10-16 20:21:21.414571,{},2,149,0


In [37]:
for table_name, data_frame in table_data.items():
    print(f"Table: {table_name}\n")
    display(data_frame[data_frame['is_submitted']==1])


Table: submission



,id,date_added,submission_json,user_id,question_id,is_submitted


In [27]:
conn.commit()
cursor.close()
conn.close()

In [39]:
conn.close()

In [22]:
import sqlite3
from bs4 import BeautifulSoup
import re

# Connect to the SQLite database
conn = sqlite3.connect('instance/site.db')
cursor = conn.cursor()

# Select the 'context' column from the 'questions' table
cursor.execute("SELECT context FROM question;")
rows = cursor.fetchall()

# Function to clean and reformat the text
def clean_and_reformat(text):
    text = re.sub(r'\\+', '', text)
    soup = BeautifulSoup(text, 'html.parser')
    utterances = soup.find_all('div', class_='utterance')
    formatted_text = []
    for utterance in utterances:
        speaker = utterance.find('b').text
        # print("SPEAKER: ", speaker)
        text = utterance.get_text()
        print("TEXT: ", text)
        formatted_text.append(f"{text}")
    return ' '.join(formatted_text)

# Update the database with cleaned data
for row in rows:
    original_text = row[0]
    print(original_text)
    cleaned_text = clean_and_reformat(original_text)
    print("CLEANED ",cleaned_text)
    # break
    cursor.execute("UPDATE question SET context = ? WHERE context = ?", (cleaned_text, original_text))

# Commit the changes and close the connection
conn.commit()
conn.close()


"\"<div class=\\\"utterance\\\"><b>Speaker 1:</b> Hello . Can I help you , ma'am ?  </div><div class=\\\"utterance\\\"><b>Speaker 2:</b>  Yes . Could you tell me how to send some money to someone with an account with a bank in Germany ?  </div>\""
TEXT:  Speaker 1: Hello . Can I help you , ma'am ?  
TEXT:  Speaker 2:  Yes . Could you tell me how to send some money to someone with an account with a bank in Germany ?  
CLEANED  Speaker 1: Hello . Can I help you , ma'am ?   Speaker 2:  Yes . Could you tell me how to send some money to someone with an account with a bank in Germany ?  
"\"<div class=\\\"utterance\\\"><b>Speaker 1:</b> Hi , I am looking for a woman named Mia .  </div><div class=\\\"utterance\\\"><b>Speaker 2:</b>  I think I am who you are looking for . You have got to be Rubin .  </div><div class=\\\"utterance\\\"><b>Speaker 1:</b>  That would be me !  </div><div class=\\\"utterance\\\"><b>Speaker 2:</b>  Rubin ! Karen and Mike have spoken about you so often that I feel lik